In [3]:
import numpy as np
def depth2PointCloud(depth_map, fx, fy, cx, cy, min_depth=0.2, max_depth=3.0, depth_scale=1000.0):
    rows, cols = depth_map.shape
    # Apply the median filter to the depth image
    depth = cv2.medianBlur(depth_map, ksize=3)
    depth = depth.astype(np.float32)/depth_scale
    # depth = np.clip(depth, min_depth, max_depth)
    # Remove the outliers
    idx = np.where(((depth.reshape(-1)<=min_depth)+(depth.reshape(-1)>=max_depth)))[0]
    
    # Create a mesh grid for the depth map
    mesh_x, mesh_y = np.meshgrid(np.arange(cols), np.arange(rows))
    np.where(depth_map > 3)
    f = (fx+fy)/2
    # Calculate 3D coordinates (X, Y, Z) from depth values
    x = (mesh_x - cx) * depth_map / f
    y = (mesh_y - cy) * depth_map / f
    z = depth_map
    point_cloud = np.stack((x, y, z), axis=-1).reshape(-1, 3)
    point_cloud = np.delete(point_cloud, idx, axis=0)
    return point_cloud

In [2]:
import pickle
with open('dataset/sep26/camera_params.pkl', 'rb') as f:
    params = pickle.load(f)

int_params = params['int_params']
ext_params = params['ext_params']
fx, fy, cx, cy = int_params['Depth']['fx'], int_params['Depth']['fy'], int_params['Depth']['cx'], int_params['Depth']['cy']

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/sep26/camera_params.pkl'

In [ ]:
import cv2
depth1 = cv2.imread('dataset/sep26/seq1/depth/1695784268680308480.png', -1)
depth2 = cv2.imread('dataset/sep26/seq1/depth/1695784281219218432.png', -1)

pc1 = depth2PointCloud(depth1,fx, fy, cx, cy).copy()
pc2 = depth2PointCloud(depth2,fx, fy, cx, cy).copy()

In [ ]:
pc1 = np.hstack([pc1, np.ones((pc1.shape[0], 1))])
pc2 = np.hstack([pc2, np.ones((pc2.shape[0], 1))])

base_T1_ef = np.loadtxt('dataset/sep26/seq1/pose/1695784268680308480.txt')
base_T2_ef = np.loadtxt('dataset/sep26/seq1/pose/1695784281219218432.txt')

rgb_T1_tag = np.loadtxt('dataset/sep26/seq1/apriltag_pose/1695784268680308480.txt')
rgb_T2_tag = np.loadtxt('dataset/sep26/seq1/apriltag_pose/1695784281219218432.txt')

tag_T1_rgb = np.linalg.inv(rgb_T1_tag)
tag_T2_rgb = np.linalg.inv(rgb_T2_tag)

depth_T_rgb = ext_params['ir1_T_rgb']
rgb_T_depth = np.linalg.inv(depth_T_rgb)

In [ ]:
tag_pc1 = ((tag_T1_rgb@rgb_T_depth@pc1.T).T)[:,0:3]
tag_pc2 = ((tag_T2_rgb@rgb_T_depth@pc2.T).T)[:,0:3]

In [ ]:
import open3d as o3d
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pc1[:,0:3])
tag_pcd1 = o3d.geometry.PointCloud()
tag_pcd1.points = o3d.utility.Vector3dVector(tag_pc1[:,0:3])

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(pc2[:,0:3])
tag_pcd2 = o3d.geometry.PointCloud()
tag_pcd2.points = o3d.utility.Vector3dVector(tag_pc2[:,0:3])

In [ ]:
o3d.visualization.draw_geometries([pcd1,pcd2])

In [ ]:
o3d.visualization.draw_geometries([tag_pcd1, tag_pcd2])

# Camera Feed

In [1]:
from FR3Py.vision.cameras import RealSenseCamera
camera = RealSenseCamera(VGA=True, enable_imu=False, enable_ir=True, emitter_enabled=True, align_to_color=False, color_fps=30, depth_fps=30)

Enabling streams for camera:  216322074479


In [2]:
K = camera.getIntrinsics()['Depth']
fx,fy,cx,cy = K['fx'], K['fy'], K['cx'], K['cy']

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/robocaster/projects/manipulation/FR3Py/FR3Py/vision/cameras.py", line 123, in _run_grab_frames
    self.grab_frames()
  File "/home/robocaster/projects/manipulation/FR3Py/FR3Py/vision/cameras.py", line 141, in grab_frames
    frames = self.pipeline.wait_for_frames()
RuntimeError: Frame didn't arrive within 5000


## Snapshot

In [8]:
import cv2
import numpy as np
import open3d as o3d
camera.grab_frames()
depth = camera.depth_frame
pc = depth2PointCloud(depth,fx, fy, cx, cy)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
o3d.visualization.draw_geometries([pcd])

RuntimeError: Frame didn't arrive within 5000

## Live

In [5]:
import cv2
import numpy as np
import time
import cv2
import numpy as np
import open3d as o3d
camera.grab_frames()
depth = camera.depth_frame
pc = depth2PointCloud(depth,fx, fy, cx, cy)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)

while True:
    time.sleep(0.01)
    camera.grab_frames()
    depth = camera.depth_frame
    pc = depth2PointCloud(depth,fx, fy, cx, cy)
    pcd.points = o3d.utility.Vector3dVector(pc)
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


RuntimeError: Frame didn't arrive within 5000